# Website link : https://www.99acres.com/

In [7]:
#Initializing driver
from selenium import webdriver
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options #For edge, firefox etc. they also have options class
from selenium.webdriver.common.action_chains import ActionChains

# options
chrome_options = Options()
chrome_options.add_argument("--disable-http2")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--enable-features=NetworkServiceInProcess")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"
)


service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options = chrome_options)
driver.maximize_window()

#Explicit wait
wait = WebDriverWait(driver, 10)

#Function for Checking if document loaded fully
def waitToLoad(driver, wait):
    page_title = driver.title
    try:
        wait.until(
            lambda d : d.execute_script("return document.readyState") == "complete"
        )
    except Exception as e:
        print("Page didn't load fully within time limit. ")
        print("Error generated :",e)
    else:
        print("Page ",page_title," loaded successfully!")



#Opening website
url = "https://www.99acres.com/"
driver.get(url)

#Check if loading is done
waitToLoad(driver, wait)

#Identifying the search bar
try:
    search_bar = wait.until(
        EC.presence_of_element_located((By.XPATH,'//*[@id="keyword2"]'))
    )
except:
    print("Couldn't find the search bar .")
else:
    search_bar.send_keys("Chennai")
    time.sleep(2)

#Selecting valid option from dropdown list after entering text
try:
    valid_option = wait.until(
        EC.element_to_be_clickable((By.XPATH,'//*[@id="0"]'))
    )
except:
    print("Option isn't clickable.")
else:
    valid_option.click()
    time.sleep(2)

#Click on search button
try:
    search_btn = wait.until(
        EC.element_to_be_clickable((By.XPATH,'//*[@id="searchform_search_btn"]'))
    )
except:
    print("Button isn't clickable.")
else:
    search_btn.click()
    waitToLoad(driver, wait)

#Adjust the budget slider
try:
    slider = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="budgetLeftFilter_max_node"]'))
    )
except:
    print("Slider isn't clickable.")
else:
    actions = ActionChains(driver)
    actions.click_and_hold(slider).move_by_offset(-73,0).release().perform()
    time.sleep(2)

#Selecting verify tag
try:
    verify = wait.until(
    EC.element_to_be_clickable((By.XPATH,"//span[normalize-space()='Verified']"))
    )
except:
    print("Can't click on verify tag.")
else:
    verify.click()
    time.sleep(1)

#Selecting ready-to-move tag
try:
    readyToMove = wait.until(
    EC.element_to_be_clickable((By.XPATH,"//span[normalize-space()='Ready To Move']"))
    )
except:
    print("Can't click on 'ready To Move' tag.")
else:
    readyToMove.click()
    time.sleep(1)

#Click on '>' tag twice
while True:
    try:
        fiter_right_btn = wait.until(
            EC.presence_of_element_located((By.XPATH, "//i[@class='iconS_Common_24 icon_upArrow cc__rightArrow']"))
        )
    except:
        print("Reached the end of tags.")
        break
    else:
        fiter_right_btn.click()
        time.sleep(1)

#Selecting 'with photos' tag
try:
    withPhoto = wait.until(
    EC.element_to_be_clickable((By.XPATH,"//span[normalize-space()='With Photos']"))
    )
except:
    print("Can't click on 'with photos' tag.")
else:
    withPhoto.click()
    time.sleep(1)

#Selecting 'with videos' tag
try:
    withVideo = wait.until(
    EC.element_to_be_clickable((By.XPATH,"//span[normalize-space()='With Videos']"))
    )
except:
    print("Can't click on 'with videos' tag.")
else:
    withVideo.click()
    time.sleep(1)

#Navigate and scrape the data
data = []
page_count = 0
while True:
	page_count += 1
	try:
		next_page_button = driver.find_element(By.XPATH, "//a[normalize-space()='Next Page >']")
	except:
		print(f"Timeout because we have navigated all the {page_count} pages.\n")
		break
	else:
		try:
			driver.execute_script("window.scrollBy(0, arguments[0].getBoundingClientRect().top - 100);", next_page_button)
			time.sleep(2)
	
			# scraping the data
			rows = driver.find_elements(By.CLASS_NAME, "tupleNew__TupleContent")
			for row in rows:
				# property name
				try:
					name = row.find_element(By.CLASS_NAME, "tupleNew__headingNrera").text
				except:
					name = np.nan

				# property location
				try:
					location = row.find_element(By.CLASS_NAME, "tupleNew__propType").text
				except:
					location = np.nan

				# property price
				try:
					price = row.find_element(By.CLASS_NAME, "tupleNew__priceValWrap").text
				except:
					price = np.nan

				# property area and bhk
				try:
					elements = row.find_elements(By.CLASS_NAME, "tupleNew__area1Type")
				except:
					area, bhk = [np.nan, np.nan]
				else:
					area, bhk = [ele.text for ele in elements]
					
				property = {
					"name": name,
					"location": location,
					"price": price,
					"area": area,
					"bhk": bhk
				}
				data.append(property)
			
			wait.until(
				EC.element_to_be_clickable((By.XPATH, "//a[normalize-space()='Next Page >']"))
			).click()
			time.sleep(5)
		except:
			print("Timeout while clicking on \"Next Page\".\n")

# scraping data from the last page
rows = driver.find_elements(By.CLASS_NAME, "tupleNew__TupleContent")
for row in rows:
	# property name
	try:
		name = row.find_element(By.CLASS_NAME, "tupleNew__headingNrera").text
	except:
		name = np.nan

	# property location
	try:
		location = row.find_element(By.CLASS_NAME, "tupleNew__propType").text
	except:
		location = np.nan

	# property price
	try:
		price = row.find_element(By.CLASS_NAME, "tupleNew__priceValWrap").text
	except:
		price = np.nan

	# property area and bhk
	try:
		elements = row.find_elements(By.CLASS_NAME, "tupleNew__area1Type")
	except:
		area, bhk = [np.nan, np.nan]
	else:
		area, bhk = [ele.text for ele in elements]
					
	property = {
		"name": name,
		"location": location,
		"price": price,
		"area": area,
		"bhk": bhk
	}
	data.append(property)

time.sleep(2)
driver.quit()

# ----- CLEANING THE DATA -----

df_properties = (
	pd
	.DataFrame(data)
	.drop_duplicates()
	.apply(lambda col: col.str.strip().str.lower() if col.dtype == "object" else col)
	.assign(
		is_starred=lambda df_: df_.name.str.contains("\n").astype(int),
		name=lambda df_: (
			df_
			.name
			.str.replace("\n[0-9.]+", "", regex=True)
			.str.strip()
			.replace("adroit district s", "adroit district's")
		),
		location=lambda df_: (
			df_
			.location
			.str.replace("chennai", "")
			.str.strip()
			.str.replace(",$", "", regex=True)
			.str.split("in")
			.str[-1]
			.str.strip()
		),
		price=lambda df_: (
			df_
			.price
			.str.replace("₹", "")
			.apply(lambda val: float(val.replace("lac", "").strip()) if "lac" in val else float(val.replace("cr", "").strip()) * 100)
		),
		area=lambda df_: (
			df_
			.area
			.str.replace("sqft", "")
			.str.strip()
			.str.replace(",", "")
			.pipe(lambda ser: pd.to_numeric(ser))
		),
		bhk=lambda df_: (
			df_
			.bhk
			.str.replace("bhk", "")
			.str.strip()
			.pipe(lambda ser: pd.to_numeric(ser))
		)
	)
	.rename(columns={
		"price": "price_lakhs",
		"area": "area_sqft"
	})
	.reset_index(drop=True)
	.to_excel("chennai-properties-99acres.xlxx", index=False)
)

            
    
                
            
        

Page  India Real Estate Property Site - Buy Sell Rent Properties Portal - 99acres.com  loaded successfully!
Page  Property in Kolkata - Real Estate in Kolkata  loaded successfully!
Reached the end of tags.
Timeout cuz we have reached the last page.
